<a href="https://colab.research.google.com/github/adititadkod15-tech/HinglishLID1/blob/main/Akasharmukha_transliteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install indic-transliteration

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.6/159.6 kB 4.7 MB/s eta 0:00:00


In [4]:
import torch
from transformers import pipeline
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate
import re

# 1. Initialize LID Pipeline on GPU
lid_pipeline = pipeline("token-classification", model="l3cube-pune/hing-bert-lid", device=0)

def make_actual_hinglish(sentence):
    if not sentence.strip():
        return ""

    # Run LID
    results = lid_pipeline(sentence)

    # Reconstruct tokens
    words_data = []
    current_word, current_label = "", ""
    for entity in results:
        raw_word = entity['word']
        label = entity['entity']
        if raw_word.startswith("##"):
            current_word += raw_word.replace("##", "")
        else:
            if current_word: words_data.append((current_word, current_label))
            current_word, current_label = raw_word, label
    if current_word: words_data.append((current_word, current_label))

    # Transliterate ONLY Hindi words
    mixed_words = []
    for word, label in words_data:
        if label == 'HI':
            # Remove symbols/numbers for the transliteration engine
            clean_word = re.sub(r'[^a-zA-Z]', '', word)
            if clean_word:
                hindi_script = transliterate(clean_word.lower(), sanscript.ITRANS, sanscript.DEVANAGARI)
                mixed_words.append(hindi_script)
            else:
                mixed_words.append(word)
        else:
            # Keep English and Punctuation as is
            mixed_words.append(word)

    return " ".join(mixed_words)

# 2. Read your .txt file and process
input_file = "/content/Hinglish_data3 (1).txt"  # <-- Change this to your uploaded file name
output_file = "mixed_hinglish_output.txt"

print(f"Reading {input_file}...")

with open(input_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()

print(f"Processing {len(lines)} lines on A100...")

processed_lines = []
for i, line in enumerate(lines):
    converted = make_actual_hinglish(line.strip())
    processed_lines.append(converted)

    # Progress update every 100 lines
    if (i + 1) % 100 == 0:
        print(f"Processed {i + 1}/{len(lines)} lines")

# 3. Save the results
with open(output_file, 'w', encoding='utf-8') as f:
    for line in processed_lines:
        f.write(line + "\n")

print(f"\nSuccess! File saved as: {output_file}")

Device set to use cuda:0


Reading /content/Hinglish_data3 (1).txt...
Processing 7155 lines on A100...


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Processed 100/7155 lines
Processed 200/7155 lines
Processed 300/7155 lines
Processed 400/7155 lines
Processed 500/7155 lines
Processed 600/7155 lines
Processed 700/7155 lines
Processed 800/7155 lines
Processed 900/7155 lines
Processed 1000/7155 lines
Processed 1100/7155 lines
Processed 1200/7155 lines
Processed 1300/7155 lines
Processed 1400/7155 lines
Processed 1500/7155 lines
Processed 1600/7155 lines
Processed 1700/7155 lines
Processed 1800/7155 lines
Processed 1900/7155 lines
Processed 2000/7155 lines
Processed 2100/7155 lines
Processed 2200/7155 lines
Processed 2300/7155 lines
Processed 2400/7155 lines
Processed 2500/7155 lines
Processed 2600/7155 lines
Processed 2700/7155 lines
Processed 2800/7155 lines
Processed 2900/7155 lines
Processed 3000/7155 lines
Processed 3100/7155 lines
Processed 3200/7155 lines
Processed 3300/7155 lines
Processed 3400/7155 lines
Processed 3500/7155 lines
Processed 3600/7155 lines
Processed 3700/7155 lines
Processed 3800/7155 lines
Processed 3900/7155 l

In [16]:
!pip install aksharamukha

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.1/183.1 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 140.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.0/532.0 kB 44.7 MB/s eta 0:00:00


In [3]:
from aksharamukha import transliterate

In [4]:
def fix_and_phonemize(text):
    clean_hindi = text.replace('् ', ' ').replace('्.', '.')
    if clean_hindi.endswith('्'):
        clean_hindi = clean_hindi[:-1]

    # STEP 2: Convert to Roman Hinglish (keeping English intact)
    # Aksharamukha's 'RomanColloquial' is best for readable Roman Hindi
    roman_hinglish = transliterate.process('Devanagari', 'RomanColloquial', clean_hindi)

    # STEP 3: Generate Phonemes (IPA)
    # We use the 'IPA' target which is built specifically for phonetic accuracy
    phonemes = transliterate.process('Devanagari', 'IPA', clean_hindi)

    return clean_hindi, roman_hinglish, phonemes

In [5]:
from aksharamukha import transliterate

def process_with_clean_spelling(text):
    # 1. First, fix your dataset's trailing halants (्)
    clean_devanagari = text.replace('् ', ' ').replace('्.', '.')
    if clean_devanagari.endswith('्'):
        clean_devanagari = clean_devanagari[:-1]

    # 2. CONVERT TO ROMAN HINDI (With Schwa Deletion enabled)
    # This acts as a spellcheck to remove the 'a' from the end of words like 'baat'
    roman_clean = transliterate.process('Devanagari', 'RomanColloquial', clean_devanagari,
                                        pre_options=['RemoveSchwaHindi'])

    # 3. GET IPA PHONEMES (Also with Schwa Deletion)
    ipa_clean = transliterate.process('Devanagari', 'IPA', clean_devanagari,
                                      pre_options=['RemoveSchwaHindi'])

    return roman_clean, ipa_clean

# --- Testing with your sentence ---
sample_sentences = ['आज् कि इस् roundtable discussion क मुख्य् विशय् बहुत् हि भवुक् है ( 999 words ) - wars और् conflicts के emotional असर् पर् चर्च कर्नि hai . ये एक् ऐसि चीज़् है जो हर् इन्सान् को छू कर् जाति है , चहे वोह् उस्मेइन् directly शामिल् हो य न ho .',
                    'पह्ले तोह् हुम् बात् कर्ते हैन् cambodian civil war कि ( 999 words ) . वहन् पर् जो हुअ , वोह् सच् मेइन् heartache और् शोक् से भर tha . लखोन् लोगोन् कि ज़िन्दगियन् बर्बद् हो गयिहि . कित्नि हि औरतेन् विध्वा हो गयिहि , और् कित्ने बच्चे अनथ् ! सब् के चेह्रोन् पर् सिर्फ़् निरश और् melancholic हि नज़र् आत होग उस् समय् .',
                    'मुझे याद् है जब् मैने एक् documentary देख थ इस् war के बारे mein . वोह् सब् सुन्कर् मेर रूह् कान्प् गय tha . बच्चोन् को अप्ने मा - बाप् को खोते देख्न , औरतोन् कि विध्ववस्थ , ये सब् देख्कर् एक्दुम् से मेर मन् उदास् और् dismay से भर् गय tha .',
                    'जैस कि आप् सब् जान्ते हैन् , wars के beech chemical और् biological weapons क इस्तेमाल् भि होत hai . इन्से तोह् और् भि नुक्सन् होत है ज़िन्दगियोन् ka . एक् missile य bomb से लखोन् लोग् दुखि हो जाते हैन् - अप्ने घर् गवा देते हैन् , अप्ने परिवार् को खा देते hain . ये सब् सोच्न हि इत्न हिदायथीन् कर्त hai .',
                    'fijian coups के बारे मेइन् पध थ मैने कुच् दिनो पेह्ले . वहन् पर् भि बहुत् कुछ् हुअ tha . आखिर् क्युन् होति हैन् ये सब् चीज़ेइन् ? क्युन् कोइ उसे bombs और् weapons कर्त है दूस्रे कि ज़िन्दगियोन् को बर्बद् कर्ने के लिये ? ये सब् जान्ने के बाद् तोह् मैन् एक्दुम् हि निम्हन्स् हो गय tha .',
                    'और् जैस कि हुम् जान्ते हैन् , india भि ऐसे हि कयि wars और् insurgencies से गुज़्र hai . जैसे कि naxalite - maoist insurgency , और् eritrean war of independence . in दोनो मेइन् कित्ने हि जवान् शहीद् हुए , और् कित्ने गरीब् लोगोन् को अप्नि ज़िन्दगियान् गवनि पदिन् . सब् के घर् बर्बद् हो gaye , सरि उमीदेन् तूत् गयिहि .']
for sample in sample_sentences:
  roman, ipa = process_with_clean_spelling(sample)

  print(f"hinglish:{sample}")
  print(f"Clean Roman: {roman}")
  print(f"Clean IPA:   {ipa}")
  print('\n')

hinglish:आज् कि इस् roundtable discussion क मुख्य् विशय् बहुत् हि भवुक् है ( 999 words ) - wars और् conflicts के emotional असर् पर् चर्च कर्नि hai . ये एक् ऐसि चीज़् है जो हर् इन्सान् को छू कर् जाति है , चहे वोह् उस्मेइन् directly शामिल् हो य न ho .
Clean Roman: aj ki is roundtable discussion ka mukhya vishay bahut hi bhavuk hai ( 999 words ) - wars aur conflicts ke emotional asar par charch karni hai . ye ek aisi chiz hai jo har insan ko chhu kar jati hai , chahe voh usmein directly shamil ho ya na ho .
Clean IPA:   ɑːd͡ʒ kɪ ɪs̪ roundtable discussion kə mukʰjə ʋɪʃəj bəɦut̪ ɦɪ bʰəʋuk ɦəɪ ( 999 words ) - wars əuɾ conflicts keː emotional əs̪əɾ pəɾ t͡ʃəɾt͡ʃ kəɾn̪ɪ h ̆ai . jeː eːk əɪs̪ɪ t͡ʃiːz ɦəɪ d͡ʒoː ɦəɾ ɪn̪s̪ɑːn̪ koː t͡ʃʰuː kəɾ d͡ʒɑːt̪ɪ ɦəɪ , t͡ʃəɦeː ʋoːɦ us̪meːɪn̪ directly ʃɑːmɪl ɦoː jə n̪ə h ̆o .


hinglish:पह्ले तोह् हुम् बात् कर्ते हैन् cambodian civil war कि ( 999 words ) . वहन् पर् जो हुअ , वोह् सच् मेइन् heartache और् शोक् से भर tha . लखोन् लोगोन् कि ज़िन्दगियन् बर्बद् हो गयिहि . 

In [5]:
!pip install indic-nlp-library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 15.4 MB/s eta 0:00:00


In [2]:
from aksharamukha import transliterate
from indicnlp.normalize.indic_normalize import DevanagariNormalizer

# Initialize the Normalizer
normalizer = DevanagariNormalizer()

def  process_with_clean_spelling(text):
    # 1. CLEAN THE DATASET ARTIFACTS
    # Remove the trailing halants you mentioned earlier
    text = text.replace('् ', ' ').replace('्.', '.')

    # 2. HINDI SPELLCHECK/NORMALIZATION
    # This fixes character-level spelling mistakes
    normalized_text = normalizer.normalize(text)

    # 3. TRANSLITERATE TO ROMAN
    # We use 'pre_options' to delete schwas and 'post_options' to ensure long vowels
    roman = transliterate.process('Devanagari', 'RomanColloquial', normalized_text,
                                  pre_options=['RemoveSchwaHindi'])

    # Quick fix for the 'baat/bat' issue:
    # Usually 'bat' is mapped to 'baat' in common Hinglish
    roman = roman.replace(' bat ', ' baat ').replace(' pahle ', ' pahale ')

    # 4. GET IPA PHONEMES
    ipa = transliterate.process('Devanagari', 'IPA', normalized_text,
                                pre_options=['RemoveSchwaHindi'])

    return roman, ipa

# --- TEST ---
sample_sentences = ['आज् कि इस् roundtable discussion क मुख्य् विशय् बहुत् हि भवुक् है ( 999 words ) - wars और् conflicts के emotional असर् पर् चर्च कर्नि hai . ये एक् ऐसि चीज़् है जो हर् इन्सान् को छू कर् जाति है , चहे वोह् उस्मेइन् directly शामिल् हो य न ho .',
                    'पह्ले तोह् हुम् बात् कर्ते हैन् cambodian civil war कि ( 999 words ) . वहन् पर् जो हुअ , वोह् सच् मेइन् heartache और् शोक् से भर tha . लखोन् लोगोन् कि ज़िन्दगियन् बर्बद् हो गयिहि . कित्नि हि औरतेन् विध्वा हो गयिहि , और् कित्ने बच्चे अनथ् ! सब् के चेह्रोन् पर् सिर्फ़् निरश और् melancholic हि नज़र् आत होग उस् समय् .',
                    'मुझे याद् है जब् मैने एक् documentary देख थ इस् war के बारे mein . वोह् सब् सुन्कर् मेर रूह् कान्प् गय tha . बच्चोन् को अप्ने मा - बाप् को खोते देख्न , औरतोन् कि विध्ववस्थ , ये सब् देख्कर् एक्दुम् से मेर मन् उदास् और् dismay से भर् गय tha .',
                    'जैस कि आप् सब् जान्ते हैन् , wars के beech chemical और् biological weapons क इस्तेमाल् भि होत hai . इन्से तोह् और् भि नुक्सन् होत है ज़िन्दगियोन् ka . एक् missile य bomb से लखोन् लोग् दुखि हो जाते हैन् - अप्ने घर् गवा देते हैन् , अप्ने परिवार् को खा देते hain . ये सब् सोच्न हि इत्न हिदायथीन् कर्त hai .',
                    'fijian coups के बारे मेइन् पध थ मैने कुच् दिनो पेह्ले . वहन् पर् भि बहुत् कुछ् हुअ tha . आखिर् क्युन् होति हैन् ये सब् चीज़ेइन् ? क्युन् कोइ उसे bombs और् weapons कर्त है दूस्रे कि ज़िन्दगियोन् को बर्बद् कर्ने के लिये ? ये सब् जान्ने के बाद् तोह् मैन् एक्दुम् हि निम्हन्स् हो गय tha .',
                    'और् जैस कि हुम् जान्ते हैन् , india भि ऐसे हि कयि wars और् insurgencies से गुज़्र hai . जैसे कि naxalite - maoist insurgency , और् eritrean war of independence . in दोनो मेइन् कित्ने हि जवान् शहीद् हुए , और् कित्ने गरीब् लोगोन् को अप्नि ज़िन्दगियान् गवनि पदिन् . सब् के घर् बर्बद् हो gaye , सरि उमीदेन् तूत् गयिहि .']
for sample in sample_sentences:
  roman, ipa = process_with_clean_spelling(sample)

  print(f"hinglish:{sample}")
  print(f"Clean Roman: {roman}")
  print(f"Clean IPA:   {ipa}")
  print('\n')

hinglish:आज् कि इस् roundtable discussion क मुख्य् विशय् बहुत् हि भवुक् है ( 999 words ) - wars और् conflicts के emotional असर् पर् चर्च कर्नि hai . ये एक् ऐसि चीज़् है जो हर् इन्सान् को छू कर् जाति है , चहे वोह् उस्मेइन् directly शामिल् हो य न ho .
Clean Roman: aj ki is roundtable discussion ka mukhya vishay bahut hi bhavuk hai ( 999 words ) - wars aur conflicts ke emotional asar par charch karni hai . ye ek aisi chiz hai jo har insan ko chhu kar jati hai , chahe voh usmein directly shamil ho ya na ho .
Clean IPA:   ɑːd͡ʒ kɪ ɪs̪ roundtable discussion kə mukʰjə ʋɪʃəj bəɦut̪ ɦɪ bʰəʋuk ɦəɪ ( 999 words ) - wars əuɾ conflicts keː emotional əs̪əɾ pəɾ t͡ʃəɾt͡ʃ kəɾn̪ɪ h ̆ai . jeː eːk əɪs̪ɪ t͡ʃiːz ɦəɪ d͡ʒoː ɦəɾ ɪn̪s̪ɑːn̪ koː t͡ʃʰuː kəɾ d͡ʒɑːt̪ɪ ɦəɪ , t͡ʃəɦeː ʋoːɦ us̪meːɪn̪ directly ʃɑːmɪl ɦoː jə n̪ə h ̆o .


hinglish:पह्ले तोह् हुम् बात् कर्ते हैन् cambodian civil war कि ( 999 words ) . वहन् पर् जो हुअ , वोह् सच् मेइन् heartache और् शोक् से भर tha . लखोन् लोगोन् कि ज़िन्दगियन् बर्बद् हो गयिहि . 